# 📊 Supply Usage Analysis: Warehouse & Retail Sales Data

### 🧩 Project Overview
This project analyzes supply usage patterns across retail and warehouse channels using the **Warehouse and Retail Sales** dataset from [Data.gov](https://data.gov).  
The goal is to uncover insights about product movement, supplier performance, and seasonal trends thereby helping inform smarter inventory management and demand planning decisions. The framework presented herein is easily adapted to other inventory datasets.

### 🎯 Objectives
- Combine retail and warehouse data to create a unified **supply usage** metric  
- Explore **trends over time** by supplier, item type, and sales channel  
- Identify **top-moving products** and periods of unusually high or low demand  
- Visualize patterns and provide data-driven recommendations for inventory optimization  

### 🧾 Dataset
This dataset includes monthly records of warehouse and retail activity with the following key fields:
- `year`, `month` – reporting period  
- `supplier`, `item_code`, `item_description`, `item_type` – product identifiers  
- `retail_sales`, `retail_transfers`, `warehouse_sales` – quantities across distribution channels  
- `date` – constructed timestamp for time-series analysis  

Data Source: [Data.gov – Warehouse and Retail Sales Dataset](https://data.gov)

### 🛠️ Tools & Techniques
This notebook uses Python and the standard data science toolkit:
- **pandas** for data cleaning and aggregation  
- **numpy** for calculations  
- **matplotlib** / **seaborn** for visualization  
- **Streamlit (optional)** for building an interactive dashboard version  

### 📈 Workflow
1. **Data Cleaning** – handle missing values, create a monthly `date` column  
2. **Feature Engineering** – calculate a total `usage` metric combining all outflow channels  
3. **Exploratory Analysis** – aggregate and visualize usage trends  
4. **Insights** – summarize supplier performance, seasonal effects, and usage variability  

### 💡 Key Takeaways
By the end of this project, you’ll be able to:
- Quantify total supply usage over time  
- Compare usage across suppliers and channels  
- Identify trends that inform stocking, procurement, and logistics strategies  

---

*Created by [Graham Kesler O'Connor](https://github.com/grahamkesler)*  
*Project Type: Exploratory Data Analysis (EDA) | Language: Python | Dataset: Public (Data.gov)*  


In [61]:
import pandas as pd
import numpy as np
df = pd.read_csv('/portfolio/data/Warehouse_and_Retail_Sales.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307645 entries, 0 to 307644
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   YEAR              307645 non-null  int64  
 1   MONTH             307645 non-null  int64  
 2   SUPPLIER          307478 non-null  object 
 3   ITEM CODE         307645 non-null  object 
 4   ITEM DESCRIPTION  307645 non-null  object 
 5   ITEM TYPE         307644 non-null  object 
 6   RETAIL SALES      307642 non-null  float64
 7   RETAIL TRANSFERS  307645 non-null  float64
 8   WAREHOUSE SALES   307645 non-null  float64
dtypes: float64(3), int64(2), object(4)
memory usage: 21.1+ MB


In [62]:
# create date column
df['date'] = pd.to_datetime(dict(year=df['YEAR'],month=df['MONTH'], day = 1))

# reformat column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ','_')

# number of duplicate rows
dups = df.duplicated().astype(int).sum()
print(dups)

# find number and percent of missing entries for each column
missing_cnt = df.isna().sum().sort_values(ascending=False)
missing_pct = (df.isna().mean()*100).round(2).sort_values(ascending=False)
audit = pd.DataFrame({"missing_n": missing_cnt, "missing_%": missing_pct})
print(audit)

# Correlate missingness with other columns (not so necessary here bc only one column has a lot of empties)
miss_flags = df.isna().astype(int).add_prefix("isna__")
corr_with_miss = miss_flags.corr()  # look for blocks/structure




0
                  missing_n  missing_%
date                      0       0.00
item_code                 0       0.00
item_description          0       0.00
item_type                 1       0.00
month                     0       0.00
retail_sales              3       0.00
retail_transfers          0       0.00
supplier                167       0.05
warehouse_sales           0       0.00
year                      0       0.00


In [75]:
# Define total usage as the sum of all outflow channels
FLOW_COLS = ['retail_sales', 'retail_transfers', 'warehouse_sales']
# if columns aren't already numeric
#df[FLOW_COLS] = df[FLOW_COLS].apply(pd.to_numeric, errors='coerce')
df['usage'] = df[FLOW_COLS].sum(axis=1).fillna(0)

## (Optional) keep a tidy view of each flow for breakdowns
# flows_long = df.melt(
#     id_vars=['date', 'supplier', 'item_code', 'item_description', 'item_type'],
#     value_vars=FLOW_COLS,
#     var_name='flow_type',
#     value_name='qty'
# )
# # flow breakdown
# flow_breakdown = (
#     flows_long
#       .groupby(['date','supplier','item_code','flow_type'], as_index=False)['qty']
#       .sum()
#       .pivot_table(index=['date','supplier','item_code'], columns='flow_type', values='qty', fill_value=0)
#       .reset_index()
# )

# Aggregate usage acrocss groups
monthly = (
    df.groupby(['supplier','item_code','item_description','item_type',
                pd.Grouper(key='date',freq='MS')],as_index=False)
    .agg(usage=('usage','sum'))
    .sort_values(['supplier','item_code','date'])
)


In [56]:
# adding trend features

# Per-item MoM change
monthly['year'] = monthly['date'].dt.year

monthly['usage_mom'] = (
    monthly
      .groupby(['supplier','item_code'])['usage']
      .pct_change()
)

# Per-item rolling 3-month average
monthly['usage_ma3'] = (
    monthly
      .groupby(['supplier','item_code'])['usage']
      .rolling(3, min_periods=1).mean()
      .reset_index(level=[0,1], drop=True)
)

# Per-item rolling YTD totals
monthly['ytd_usage'] = (
    monthly
      .groupby(['supplier','item_code','year'])['usage']
      .cumsum()
)

In [46]:
# top moving items from previous month
last_month = monthly['date'].max()
top_items = monthly[monthly['date']==last_month].nlargest(20,'usage')[['supplier','item_code','item_description','usage']]


In [47]:
# pivot table of supplier usages over time
supplier_pivot = (
    monthly
      .pivot_table(index='date', columns='supplier', values='usage', aggfunc='sum')
      .fillna(0)
)

supplier_pivot.head()

supplier,8 VINI INC,A HARDY USA LTD,A I G WINE & SPIRITS,A VINTNERS SELECTIONS,A&E INC,A&W BORDERS LLC,ADAMBA IMPORTS INTL,AIKO IMPORTERS INC,ALLAGASH BREWING COMPANY,ALLIED IMPORTERS USA LTD,...,WILLIAMS CORNER WINE,WILSON DANIELS LTD,WINE BRIDGE IMPORTS INC,WINEBOW INC,WINERY EXCHANGE INC,WITH MALUS AFORETHOUGHT LLC,YOUNG WON TRADING INC,YUENGLING BREWERY,Z WINE GALLERY IMPORTS LLC,ZURENA LLC
date,,,,,,,,,,,,,,,,,,,,,
2017-06-01,0.08,0.16,18.22,5044.07,0.98,0.16,2.18,1.46,301.66,1.85,...,5.00,129.85,64.16,0.00,0.0,0.0,481.20,9863.97,7.00,15.87
2017-07-01,0.08,0.16,14.66,4327.58,1.04,0.58,2.17,1.51,264.28,1.02,...,10.16,98.74,85.14,-0.01,0.0,0.0,475.18,8772.87,10.00,5.91
2017-08-01,0.24,0.16,18.23,4699.89,1.82,0.24,2.19,0.82,311.21,0.68,...,4.08,101.75,76.37,-0.50,0.0,0.0,512.83,9020.93,6.72,5.11
2017-09-01,0.08,0.00,25.34,4608.99,0.16,0.32,18.51,0.25,233.25,5.35,...,22.00,41.52,98.07,0.25,0.0,0.0,469.19,7886.30,3.55,4.04
2017-10-01,3.16,0.00,8.25,4884.37,0.91,0.00,7.54,5.49,247.81,5.51,...,5.00,51.82,64.75,0.25,0.0,0.0,588.08,7815.80,1.65,0.97


In [57]:
# Z-score within each item
monthly['usage_z'] = (
    monthly.groupby(['supplier','item_code'])['usage']
           .transform(lambda s: (s - s.mean()) / (s.std(ddof=0) or np.nan))
)
monthly['is_spike'] = monthly['usage_z'].abs() >= 3

num_spikes = monthly['is_spike'].sum()
monthly_spikes = monthly[monthly['is_spike']==True]

In [77]:
smalldf = df.nsmallest(10,'usage')
smalldf

,year,month,supplier,item_code,item_description,item_type,retail_sales,retail_transfers,warehouse_sales,date,usage
254066,2019,8,NaN,600,ISTORE TRAINING OVERVIEW,REF,0.0,0.0,-7800.0,2019-08-01,-7800.0
89467,2017,10,ANHEUSER BUSCH INC,175,EMPTY 1/2 KEG (30.00),DUNNAGE,0.0,0.0,-4996.0,2017-10-01,-4996.0
35897,2017,6,ANHEUSER BUSCH INC,175,EMPTY 1/2 KEG (30.00),DUNNAGE,0.0,0.0,-4804.0,2017-06-01,-4804.0
62504,2017,8,ANHEUSER BUSCH INC,175,EMPTY 1/2 KEG (30.00),DUNNAGE,0.0,0.0,-4536.0,2017-08-01,-4536.0
271028,2019,10,PREMIUM DISTRIBUTORS INC,175,EMPTY 1/2 KEG (30.00),DUNNAGE,0.0,0.0,-4365.0,2019-10-01,-4365.0
49468,2017,7,ANHEUSER BUSCH INC,175,EMPTY 1/2 KEG (30.00),DUNNAGE,0.0,0.0,-4364.0,2017-07-01,-4364.0
233700,2019,7,PREMIUM DISTRIBUTORS INC,175,EMPTY 1/2 KEG (30.00),DUNNAGE,0.0,0.0,-4349.0,2019-07-01,-4349.0
103113,2017,11,ANHEUSER BUSCH INC,175,EMPTY 1/2 KEG (30.00),DUNNAGE,0.0,0.0,-4312.0,2017-11-01,-4312.0
132024,2018,1,ANHEUSER BUSCH INC,175,EMPTY 1/2 KEG (30.00),DUNNAGE,0.0,0.0,-4302.0,2018-01-01,-4302.0
195730,2019,4,PREMIUM DISTRIBUTORS INC,175,EMPTY 1/2 KEG (30.00),DUNNAGE,0.0,0.0,-4299.0,2019-04-01,-4299.0
